Step 1 is to setup the Google collab environment 

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2 
!nvcc --version
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git
%load_ext nvcc_plugin
from google.colab import drive
drive.mount("/content/gdrive")







In [ ]:
%%writefile main.cu
#include "/usr/include/opencv2/opencv.hpp"
#include <iostream>
#include <vector>
using namespace cv;
using namespace std;

__global__ void to_grayscale(uchar* d_image, uchar* d_gray_image) { 

      int index = blockIdx.x;
      
      int red = d_image[index * 3 + 0];
      int green = d_image[index * 3 + 1];
      int blue = d_image[index * 3 + 2];
      int new_pixel = (0.299 * red) + (0.587 * green) + (0.114 * blue);
     
      d_gray_image[index] = new_pixel;
}


int main(int argc, char** argv) {
    
  Mat h_input_image = imread("/content/gdrive/MyDrive/wec.jpg");
  Mat h_im_gray = imread("/content/gdrive/MyDrive/wec.jpg", CV_LOAD_IMAGE_GRAYSCALE);
  
  // Convert the OpenCV matrixes to vectors
  vector<uchar> h_input_image_array(h_input_image.rows * h_input_image.cols * h_input_image.channels());
  h_input_image_array.assign(h_input_image.data, h_input_image.data + h_input_image.total() * h_input_image.channels());

  vector<uchar> h_im_gray_array(h_im_gray.rows * h_im_gray.cols * h_im_gray.channels());
  h_im_gray_array.assign(h_im_gray.data, h_im_gray.data + h_im_gray.total() * h_im_gray.channels());
 
  uchar* d_image;
  uchar* d_im_gray;

  size_t color_sizeInBytes = h_input_image_array.size() * sizeof(uchar);
  size_t grayscale_sizeInBytes = h_im_gray_array.size() * sizeof(uchar);

  cudaMalloc((void**)&d_image, color_sizeInBytes);
  cudaMalloc((void**)&d_im_gray, grayscale_sizeInBytes);
 
  cudaMemcpy(d_image, h_input_image_array.data(), color_sizeInBytes, cudaMemcpyHostToDevice);
  
  dim3 grid(h_input_image.rows * h_input_image.cols, 1, 1);
  to_grayscale<<<grid, 1>>>(d_image, d_im_gray);

  cudaMemcpy(h_im_gray.data, d_im_gray, grayscale_sizeInBytes, cudaMemcpyDeviceToHost);
  imwrite("/content/gdrive/MyDrive/processed_image.jpg", h_im_gray);
}

In [ ]:
%%script bash
nvcc main.cu -o main.cu `pkg-config --cflags --libs opencv`

In [ ]:
%%script bash
./main.cu

In [ ]:
import cv2 as cv
import matplotlib.pyplot as plt
img = cv.imread('/content/gdrive/MyDrive/processed_image.jpg')
plt.imshow(img)

/* Define the kernel function*/


/* Load an image */

/* Allocate storage on the GPU */
/* CPU copies data from CPU -> GPU */
/* Launch kernels*/
/* CPU copies results from the GPU back to the CPU */









